# GRAPE on a qubit with an uncertain frequency

Lets start with the example of GRAPE on a qubit, optimizing the drive envelope to be robust to variations in qubit frequency. This example will illustrate the utility of batching, allowing us to average over many different qubit frequencies to obtain a pulse that achieves a high-fidelity gate and is robust to frequency fluctuations.

This example is available as a Jupyter notebook [here](https://github.com/dkweiss31/qontrol/blob/main/docs/examples/qubit.ipynb).

In [1]:
import dynamiqs as dq
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import optax
from jax import Array
from jax.random import PRNGKey, normal

import qontrol as ql

In [2]:
time = 30
control_dt = 2.0
ntimes = int(time // control_dt) + 1
optimizer = optax.adam(learning_rate=0.001, b1=0.99, b2=0.99)
tsave = jnp.linspace(0, time, ntimes)
options = ql.OptimizerOptions(
    verbose=False,
    save_states=False,
    progress_meter=None,
    epochs=4000,
    plot=True,
    plot_period=200,
)

Here we initialize the random qubit frequency fluctuations pulled from a normal distribution. Note that the drive is on resonance with the average value of the qubit frequency.

In [3]:
key = PRNGKey(42)
random_freqs = 2.0 * jnp.pi * normal(key, shape=(21,)) / 500
H0 = random_freqs[:, None, None] * dq.sigmaz()[None]
H1s = [dq.sigmax(), dq.sigmay()]
H1_labels = ['X', 'Y']

Here we define what final states the initial states should map to. In this case we want to achieve a Y gate

In [4]:
initial_states = [dq.basis(2, 0), dq.basis(2, 1)]
target_states = [-1j * dq.basis(2, 1), 1j * dq.basis(2, 0)]

We next initialize our first guess for the controls and define the function that, given the controls, returns the Hamiltonian.

In [5]:
init_drive_params = -0.001 * jnp.ones((len(H1s), ntimes - 1))


def H_pwc(values: Array) -> dq.TimeArray:
    H = H0
    for idx, _H1 in enumerate(H1s):
        H += dq.pwc(tsave, values[idx], _H1)
    return H


sesolve_model = ql.sesolve_model(H_pwc, initial_states, tsave)

In this example we use the coherent definition of the infidelity and penalize drive strengths above 16 MHz

In [6]:
costs = ql.coherent_infidelity(target_states=target_states, target_cost=0.001)
costs += ql.control_norm(2.0 * jnp.pi * 0.016, target_cost=0.1)

In [7]:
opt_params = ql.optimize(
    init_drive_params, costs, sesolve_model, optimizer=optimizer, options=options
)

optimization terminated after 1022 epochs; 
average epoch time (excluding jit) of 0.0025 s; 
max epoch time of 0.009 s; 
min epoch time of 0.002 s


We see that despite the MHz-level frequency variations of the qubit frequency, the obtained pulse succesfully performs the desired state transfer! Here we plot the population in the $|1\rangle$ state when beginning in the $|0\rangle$ state for the different batch instances

In [8]:
H = H_pwc(opt_params)
finer_times = jnp.linspace(0.0, time, 201)
plot_result = dq.sesolve(
    H,
    initial_states,
    finer_times,
    exp_ops=[dq.basis(2, idx) @ dq.dag(dq.basis(2, idx)) for idx in range(2)],
)
init_labels = [r'$|0\rangle$', r'$|1\rangle$']
exp_labels = [r'$|0\rangle$', r'$|1\rangle$']

# for brevity only plot one initial state
state_idx_to_plot = 0
fig, ax = plt.subplots()
fig.patch.set_alpha(0.1)
ax.set_facecolor('none')
expects = plot_result.expects[:, state_idx_to_plot, 1]
for e_result in expects:
    plt.plot(finer_times, np.real(e_result))
ax.set_xlabel('time [ns]')
ax.set_ylabel('$P_1$')
ax.set_title(f'initial state={init_labels[state_idx_to_plot]}')
plt.show()

|██████████| 100.0% ◆ elapsed 3.53ms ◆ remaining 0.00ms
